In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from platform import python_version
import tensorflow as tf
import os
import numpy as np

2024-06-22 05:06:45.899917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 05:06:45.900078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 05:06:46.082923: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
print(f"Python version : {python_version()}")
print(f"Tensorflow Version : {tf.__version__}")
print(f"Eager Execution: {tf.executing_eagerly()}")

Python version : 3.10.13
Tensorflow Version : 2.15.0
Eager Execution: True


In [3]:
a = tf.ones((2,2))
b = tf.zeros((2,2))
print(a,b)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32) tf.Tensor(
[[0. 0.]
 [0. 0.]], shape=(2, 2), dtype=float32)


In [4]:
out = tf.math.reduce_sum(a, axis = 1)
print(out)
c = out.get_shape()
print(c)
print(c[0])

tf.Tensor([2. 2.], shape=(2,), dtype=float32)
(2,)
2


In [5]:
a_ = tf.reshape(a, (1,4))
print(a_)
n_a = a.numpy()
print(n_a)

tf.Tensor([[1. 1. 1. 1.]], shape=(1, 4), dtype=float32)
[[1. 1.]
 [1. 1.]]


### Eager executive vs Graph Execution

In [6]:
import time
def e_matmul(mat_ea, mat_eb):
    return tf.matmul(mat_ea, mat_eb)

mat_ea = tf.random.normal((1000, 1000))
mat_eb = tf.random.normal((1000, 1000))

e_start = time.time()

result = e_matmul(mat_ea, mat_eb)

e_elapsed_time = time.time() - e_start

print(f"Total time by eager function is {e_elapsed_time: .4f}")

Total time by eager function is  0.0861


In [7]:
@tf.function
def g_matmul(mat_ga, mat_gb):
    return tf.matmul(mat_ga, mat_gb)

mat_ga = tf.random.normal((1000, 1000))
mat_gb = tf.random.normal((1000, 1000))

g_start = time.time()

result = g_matmul(mat_ga, mat_gb)

g_elapsed_time = time.time() - g_start

print(f"Total time by Graph function is {g_elapsed_time: .4f}")

Total time by Graph function is  0.1084


In [8]:
import timeit
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape = (28, 28)))
model.add(Flatten())
model.add(Dense(256, 'relu'))
model.add(Dense(128, 'relu'))
model.add(Dense(256, 'relu'))
model.add(Dense(10, 'softmax'))

eager_model = model
X = tf.random.uniform([1000, 28, 28])
print("Eager Time", timeit.timeit(lambda: eager_model(X,training=False), number=1000))

Eager Time 14.24119615799998


In [9]:
graph_model = tf.function(model)
print("Graph time:", timeit.timeit(lambda: graph_model(X,training=False), number=1000))

Graph time: 7.832199704000004


### Variables

In [10]:
tf_var = tf.Variable([5, 10])
print(f"Initial Values are {tf_var.numpy()}")

tf_var.assign([2, 2])
print(f"New Values are {tf_var.numpy()}")
      

Initial Values are [ 5 10]
New Values are [2 2]


### Convert numpy to tensor

In [11]:
nt = np.random.rand(10,6)
print(type(nt))
print(nt)

tf_nt = tf.convert_to_tensor(nt)
print(type(tf_nt))
print(tf_nt)

<class 'numpy.ndarray'>
[[0.21253143 0.40827055 0.65015297 0.38245484 0.52572016 0.86486501]
 [0.90960228 0.18800018 0.04642814 0.88114282 0.82701476 0.18818495]
 [0.64605784 0.69772349 0.00182949 0.00813264 0.21332097 0.27765145]
 [0.71025087 0.66085052 0.09019046 0.03372906 0.54279833 0.85364573]
 [0.63071268 0.38066402 0.24685708 0.98465155 0.24533286 0.27184156]
 [0.0737532  0.64625815 0.72933141 0.26545471 0.51865612 0.5245681 ]
 [0.01635456 0.81081788 0.66008251 0.79575744 0.9569023  0.15207503]
 [0.36969157 0.71428821 0.80330179 0.193411   0.00472932 0.35489632]
 [0.4798639  0.99334321 0.97940756 0.86641266 0.33229303 0.22210016]
 [0.40461849 0.18314555 0.56434579 0.69377483 0.6306337  0.421069  ]]
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[0.21253143 0.40827055 0.65015297 0.38245484 0.52572016 0.86486501]
 [0.90960228 0.18800018 0.04642814 0.88114282 0.82701476 0.18818495]
 [0.64605784 0.69772349 0.00182949 0.00813264 0.21332097 0.27765145]
 [0.71025087 

### Computing Gradient

In [12]:
import matplotlib.pyplot as plt

x = tf.Variable(2.0)

with tf.GradientTape() as tape:
    y = x**3
    
dydx = tape.gradient(y, x)

print(dydx.numpy())




12.0


In [13]:
def graph_sigmoid(x):
    x = tf.cast(x, dtype=tf.float32)
    with tf.GradientTape() as tape:
        y = 1 / (1 + tf.exp(-x))
    return tape.gradient(y, x)

val = graph_sigmoid(15)

print(val)

None


## XOR Implementation

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model, layers


class MLP(Model):
    def __init__(self, n_hidden = 2, n_out = 1):
        super(MLP, self).__init__()
        self.fc1 = layers.Dense(n_hidden,
                                activation = tf.nn.sigmoid,
                                name = 'hidden')
        self.out = layers.Dense(n_out,
                                activation = tf.nn.sigmoid,
                               name = 'out')
        
    def call(self, x):
        x = self.fc1(x)
        x = self.out(x)
        return x
    
model = MLP(n_hidden = 2, n_out = 1)

model.summary()

Model: "mlp_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out (Dense)                     │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
model_graph = tf.function(model)
learning_rate = 0.01

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate)
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits = False)

XOR_X = [ [0, 0], [0, 1], [1, 0], [1, 1]]
XOR_Y = [ [0], [1], [1], [0] ]

x_ = tf.constant(np.array(XOR_X))
y_ = tf.constant(np.array(XOR_Y))

num_epochs = 100000

In [19]:
for i in range(num_epochs):
    with tf.GradientTape() as tape:
        y_pred = model_graph(x_)
        loss = loss_fn(y_, y_pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    if(i% 10000 == 0):
        print(f"Epoch: {i}, loss : {loss.numpy()}")

print(f"Final Prediction: {model_graph(x_).numpy()}")


Epoch: 0, loss : 0.6933580636978149
Epoch: 10000, loss : 0.6931301355361938
Epoch: 20000, loss : 0.6929551959037781
Epoch: 30000, loss : 0.6926029324531555
Epoch: 40000, loss : 0.6914077997207642
Epoch: 50000, loss : 0.6843461394309998
Epoch: 60000, loss : 0.6350721120834351
Epoch: 70000, loss : 0.5400093793869019
Epoch: 80000, loss : 0.3284410536289215
Epoch: 90000, loss : 0.14525778591632843
Final Prediction: [[0.07576477]
 [0.91087264]
 [0.90044147]
 [0.06481116]]


In [22]:
import tensorflow as tf
import numpy as np
import sklearn
from sklearn.datasets import fetch_california_housing

In [23]:
def read_infile():
    calif = fetch_california_housing()
    features = np.array(calif['data'])
    target = np.array(calif['target'])
    return features, target

def feature_normalization(X, y):
    mu = np.mean(X, axis = 0)
    std = np.std(X, axis = 0)
    return (X - mu)/std

In [25]:
features, target = read_infile()
# print(features, target)
# features = features.astype(int)
# z_features = feature_normalization(features, target)
# num_features = z_features.shape[1]
# print(z_features[0:5])
print(features)
# print(num_features)

HTTPError: HTTP Error 403: Forbidden

In [26]:
from sklearn.datasets import fetch_california_housing

# Fetch the dataset
california_housing = fetch_california_housing()

HTTPError: HTTP Error 403: Forbidden